# Simulated Annealing for a 3D Skyrmion Lattice

## TODOS

* Check indices throughout code, either switch fully to mod and smaller grid or avoid mod
* Read and think about parallelization
* Setup hdf5 output of complete configuration at snapshots
* Put analysis in place: autocorrelation, phase diagram, etc.
* Best way to setup parameterfile that i can control from the makefile?

In [ ]:
using PyCall
using PyPlot
using HDF5
using StatsBase
# using ParallelAccelerator
# using Distributions
using Base.Profile
using Base.Test
using ProgressMeter
# @pyimport matplotlib.animation as anim

## Parameters & Initialization

In [ ]:
srand(123);

In [ ]:
type Parameters
    N::Int64 # number of lattice points in each direction
    Ntot::Int64 # total number of lattice points
    Nsweeps::Int64 # number of sweeps
    Nconfig::Int64 # number of configurations to save
    T::Float64 # the current temperature
    Tmu::Float64 # the reduction factor of the temperature
    Tmin::Float64 # the minimal temperature at which we stop
    J::Float64 # ferromagnetic exchange
    Jp::Float64 # ferromagnetic exchange of correction term
    K::Float64 # DM interaction
    Kp::Float64 # DM interaction of correction term
    B::Float64 # magnetic field in 3 direction
    function Parameters(; N::Int64=10, Nsweeps::Int64=30, Nconfig::Int64=100, T::Float64=1.0, Tmu::Float64=1.05, Tmin::Float64=1.0e-8, J::Float64=1.0, K::Float64=tan(2pi/10.0), B::Float64=0.0)
        Ntot = N^3
        Jp = J/16.0
        Kp = K/8.0
        new(N, Ntot, Nsweeps, Nconfig, T, Tmu, Tmin, J, Jp, K, Kp, B)
    end
end

In [ ]:
type Monitor
    reject::Int64 # number of rejected steps
    accept::Int64 # number of accepted steps
    Monitor(; reject::Int64=0, accept::Int64=0) = new(reject, accept)
end

In [ ]:
type Results
    E::Vector{Float64} # total energy
    Etmp::Float64 # current total energy
    M::Array{Float64,2} # total magnetization
    Mtmp::Vector{Float64} # current total magnetization
    T::Vector{Float64} # temperature
    idx::Int64 # current index (monte carlo time)
    Results(pars::Parameters) = new(zeros(Float64, pars.Nconfig+1), 0.0, zeros(Float64, 3, pars.Nconfig+1), zeros(Float64, 3), zeros(Float64, pars.Nconfig+1), 0)
end

In [ ]:
function uniformS2(dims::Tuple{Int64,Int64,Int64})
    phi = 2pi .* rand(dims)
    u = 2rand(dims) - 1.0
    squ = sqrt(1.0 - u.*u)
    x = squ .* cos(phi)
    y = squ .* sin(phi)
    arr = zeros(3,dims...)
        for i=1:*(dims...)
        @inbounds arr[:,i] = [x[i] y[i] u[i]]
    end
    info("Initialized random grid.")
    arr
end

@inline function uniformS2!(s::Vector{Float64})
    phi = 2pi*rand()
    u = 2rand() - 1.0
    squ = sqrt(1.0 - u*u)
    s[1] = squ*cos(phi)
    s[2] = squ*sin(phi)
    s[3] = u
    nothing
end

In [ ]:
function enforcePeriodicity!(grid::Array{Float64,4})
    N = pars.N::Int64
    for i = 1:2
        grid[:,N+i,:,:] = grid[:,i,:,:]
        grid[:,:,N+i,:] = grid[:,:,i,:]
        grid[:,:,:,N+i] = grid[:,:,:,i]
    end
    nothing
end

function enforcePeriodicity!(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64)
    N = pars.N::Int64
    for os = 1:2
        grid[:,N+os,j,k] = grid[:,os,j,k]
        grid[:,j,N+os,k] = grid[:,i,os,k]
        grid[:,i,j,N+os] = grid[:,i,j,os]
    end
    nothing
end

In [ ]:
function isPeriodic(grid::Array{Float64,4})
    N = pars.N::Int64
    if grid[:,N+1,:,:] != grid[:,1,:,:] || grid[:,:,N+1,:] != grid[:,:,1,:] ||
       grid[:,:,:,N+1] != grid[:,:,:,1] || grid[:,N+2,:,:] != grid[:,2,:,:] ||
       grid[:,:,N+2,:] != grid[:,:,2,:] || grid[:,:,:,N+2] != grid[:,:,:,2]
        return false
    end
    true
end

In [ ]:
function initRandomGrid(dims::Tuple{Int64,Int64,Int64})
    grid = uniformS2(dims)
    enforcePeriodicity!(grid)
    res.Etmp = totalEnergy(grid)
    res.Mtmp = totalMagnetization(grid)
    grid
end

In [ ]:
function init(pars::Parameters)
    (Monitor(), Results(pars))
end

function init()
    pars = Parameters()
    (pars, init(pars)...)
end

## Helper Functions

In [ ]:
function ensurePeriodicity!(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64)
    i < 3 && (grid[:,i+N,j,k] = grid[:,i,j,k])
    j < 3 && (grid[:,i,j+N,k] = grid[:,i,j,k])
    k < 3 && (grid[:,i,j,k+N] = grid[:,i,j,k])
end

In [ ]:
@inline function normalize!(grid::Array{Float64,4}, idx::Int64)
    grid[:,idx] /= norm(grid[:,idx])
end

function normalize!(grid::Array{Float64,4})
    for idx=1:pars.Ntot::Int64
        @inbounds normalize!(grid, idx)
    end
end

In [ ]:
@inline crossX(a::Vector{Float64}, b::Vector{Float64}) = -a[3]*b[2] + a[2]*b[3]
@inline crossY(a::Vector{Float64}, b::Vector{Float64}) =  a[3]*b[1] - a[1]*b[3]
@inline crossZ(a::Vector{Float64}, b::Vector{Float64}) = -a[2]*b[1] + a[1]*b[2]

In [ ]:
@inline function DMterm(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64, os::Int64)
    crossX(grid[:,i,j,k], grid[:,i+os,j,k]) +
    crossY(grid[:,i,j,k], grid[:,i,j+os,k]) +
    crossZ(grid[:,i,j,k], grid[:,i,j,k+os])
end

In [ ]:
@inline function DMtermdiff(grid::Array{Float64,4}, sdiff::Vector{Float64}, i::Int64, j::Int64, k::Int64, os::Int64)
    N = pars.N::Int64
    crossX(sdiff, grid[:,i+os,j,k] - grid[:,mod(i-os-1,N)+1,j,k]) +
    crossY(sdiff, grid[:,i,j+os,k] - grid[:,i,mod(j-os-1,N)+1,k]) +
    crossZ(sdiff, grid[:,i,j,k+os] - grid[:,i,j,mod(k-os-1,N)+1])
end

In [ ]:
@inline function FMcollect(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64, os::Int64)
    N = pars.N::Int64
    grid[:,mod(i+os-1,N)+1,j,k] + grid[:,i,mod(j+os-1,N)+1,k] + grid[:,i,j,mod(k+os-1,N)+1]
end

In [ ]:
@inline function FMterm(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64, os::Int64)
    dot(grid[:,i,j,k], FMcollect(grid, i, j, k, os))
end

## Output

In [ ]:
function getPath(filename::ASCIIString)
    datadir = "../../data/"
    string(datadir, filename, ".h5")
#     string(filename, ".h5")
end

In [ ]:
function writeArray(data::Array{Float64}, filename::ASCIIString, dataset::ASCIIString=filename)
    h5write(getPath(filename), dataset, data)
end

In [ ]:
function writeGrid(grid::Array{Float64,4}, filename::ASCIIString)
    writeArray(grid[:, 1:pars.N, 1:pars.N, 1:pars.N], filename, dataset="grid")
end

In [ ]:
@inline function snapshot()
    res.idx::Int64 += 1
    res.E[res.idx] = res.Etmp::Float64
    res.M[:,res.idx] = res.Mtmp::Vector{Float64}
    res.T[res.idx] = pars.T::Float64
end

## Simulated Annealing

In [ ]:
@inline stepT!() = (pars.T::Float64 /= pars.Tmu::Float64)

In [ ]:
@inline acceptProb(del::Float64, T::Float64) = exp(-del/T)

In [ ]:
function totalEnergy(grid::Array{Float64,4})
    E = 0.0
    N = pars.N::Int64
    for k=1:N, j=1:N, i=1:N
        E += -pars.J * FMterm(grid, i, j, k, 1) +
             pars.Jp * FMterm(grid, i, j, k, 2) -
             pars.B * grid[3,i,j,k] -
             pars.K  * DMterm(grid, i, j, k, 1) +
             pars.Kp * DMterm(grid, i, j, k, 2)
    end
    E
end

In [ ]:
function totalMagnetization(grid::Array{Float64,4})
    M = zeros(Float64, 3)
    for idx=1:pars.Ntot::Int64
        @inbounds M += grid[:,idx]
    end
    M
end

In [ ]:
function energyChange(grid::Array{Float64,4}, snew::Vector{Float64}, i::Int64, j::Int64, k::Int64)
    sdiff = snew - grid[:,i,j,k]
    -pars.B::Float64 * sdiff[3] -
    pars.J::Float64 * dot(sdiff, FMcollect(grid, i, j, k, 1) + FMcollect(grid, i, j, k, -1)) +
    pars.Jp::Float64 * dot(sdiff, FMcollect(grid, i, j, k, 2) + FMcollect(grid, i, j, k, -2)) -
    pars.K::Float64 * DMtermdiff(grid, sdiff, i, j, k, 1) +
    pars.Kp::Float64 * DMtermdiff(grid, sdiff, i, j, k, 2)
end

In [ ]:
function acceptMove!(grid::Array{Float64,4}, snew::Vector{Float64}, dE::Float64, i::Int64, j::Int64, k::Int64)
    mon.accept::Int64 += 1
    res.Etmp::Float64 += dE
    res.Mtmp::Vector{Float64} += snew - grid[:,i,j,k]
    grid[:,i,j,k] = snew
    ensurePeriodicity!(grid, i, j, k)
end

In [ ]:
function update!(grid::Array{Float64,4}, snew::Vector{Float64})
    i,j,k = rand(1:pars.N::Int64, 3)
    uniformS2!(snew)
    dE = energyChange(grid, snew, i, j, k)
    if dE < 0.0 || acceptProb(dE, pars.T::Float64) >= rand()
        acceptMove!(grid, snew, dE, i, j, k)
    else
        mon.reject::Int64 += 1
    end
end

In [ ]:
@inline function nextsweep!(grid::Array{Float64,4}, snew::Vector{Float64})
    for i=1:pars.Ntot::Int64
        update!(grid, snew)
    end
end

In [ ]:
function nextconfig!(grid::Array{Float64,4})
    snew = zeros(Float64, 3)
    for j=1:pars.Nsweeps::Int64
        nextsweep!(grid, snew)
    end
end

In [ ]:
function thermalize!(grid::Array{Float64,4})
    snew = zeros(Float64, 3)
    snapshot()
    @showprogress 1 "Thermalizing..." for i=1:pars.Nconfig::Int64
        nextsweep!(grid, snew)
        snapshot()
    end
    grid
end

In [ ]:
function run(grid::Array{Float64,4})
    snapshot()
    @showprogress 1 "Main run..." for i = 1:pars.Nconfig::Int64
        nextconfig!(grid)
        snapshot()
        stepT!()
        pars.T < pars.Tmin::Float64 && (info("Cooled below minimal temperature: $(pars.Tmin)"); break)
    end
    grid
end

function run()
    dims = (pars.N+2, pars.N+2, pars.N+2)
    grid = initRandomGrid(dims)
    run(grid)
    grid
end

## Postprocessing

In [ ]:
@inline averageSpins(M::Array{Float64,2}) = M/pars.Ntot

@inline averageSpins!(M::Array{Float64,2}) = (M /= pars.Ntot; nothing)

In [ ]:
@inline getX(grid::Array{Float64}) = squeeze(grid[1,:],1)
@inline getY(grid::Array{Float64}) = squeeze(grid[2,:],1)
@inline getZ(grid::Array{Float64}) = squeeze(grid[3,:],1)

In [ ]:
function avgConfigs()
end

In [ ]:
function braggIntensity(S::Array{Float64,2})
end

In [ ]:
function fft3(f::Array{Float64,3}, N::Int64)
    FFTW.set_num_threads(4)
    fft(f,(N, N, N))
end

In [ ]:
function variance_meansq(f::Array{Float64}; mean=nothing, meansq=nothing)
    if meansq == nothing
        if mean == nothing
            mean::Float64 = Base.mean(f)
        end
        meansq = mean*mean
    end
    (sumabs2(f)/length(f) - meansq, meansq)
end

In [ ]:
function autocorr(f::Vector{Float64})
    meansq, var = variance_meansq(f)
    N = length(f)
    auto = zeros(Float64, N)
    for t=0:N-1
        tmp = 0.0
        for i=1:N-t
            tmp += f[i]*f[i+t]
        end
        auto[t+1] = (tmp/(N-t) - meansq) / var
    end
    auto
end

In [ ]:
function intAutoTime(f::Vector{Float64})
    0.5 + sum(autocorr(f)[2:end])
end

## Run the Simulation

In [ ]:
pars = Parameters(N=30, Nsweeps=30, Nconfig=10000, T=0.0)
mon, res = init(pars);

In [ ]:
pars.T, pars.B, pars.J, pars.K, pars.N, pars.Nconfig

In [ ]:
@time grid = thermalize!(initRandomGrid((pars.N+2,pars.N+2,pars.N+2)));

In [ ]:
writeArray(grid, "T0_N30", "grid")

In [ ]:
intAutoTime(res.E)

In [ ]:
plot(res.E)

In [ ]:
plot(autocor(res.E))

In [ ]:
plot(autocorr(res.E)[1:500])

In [ ]:
plot(autocor(res.E, 0:length(res.E)-1)[1:500])

In [ ]:
plot(autocor(res.E, 0:length(res.E)-1))

In [ ]:
# Profile.clear()
# Profile.init()
@time run();

In [ ]:
# data, lidict = Profile.retrieve();

## Tests

In [ ]:
function testuniformS2()
    N = 50
    dims = (N, N, N)
    grid = uniformS2(dims);
    for i=1:*(dims...)
        @test norm(grid[:,i]) ≈ 1.0
    end
    println("finished test: uniform points on S2")
end

In [ ]:
function test()
    testuniformS2()
    println("If you do not see any errors, all tests passed.")
end
@time test()